# Matrix Analysis 2022 - EE312

## Week 3 - Signal restoration using projections
[LTS2](https://lts2.epfl.ch)

Let us consider a signal with $N$ elements, i.e. a vector in $\mathbb{R}^N$. 
Under our observations conditions, we can only recover partially the signal's values, the other remain unknown, i.e. we know that:

$x[k] = x_k$ for $k\in E = \{e_0, e_1, \dots,e_{M-1}\}$, with $E\in\mathbb{N}^M$ and $e_i\neq e_j \forall i\neq j$ (i.e. each known index is unique).

We also know that the signal is contained within lower frequencies of the spectrum. This can be expressed using the (normalized) Fourier matrix you have constructed last week $\hat{W}$.

Let $X = \hat{W}x$, $X$ is the *discrete Fourier transform* of $x$. The frequency condition can then be rewritten as 

$X[k] = 0$ for $w_c < k\leq N-w_c$.

In this notebook, we will try to reconstruct the signal by projecting its observation successively on the Fourier subspace defined above, then back to its original domain (with the constraint regarding its values), then on the Fourier domain, etc. This is a simplified version of a more general method called ["Projection onto convex sets" (POCS)](https://en.wikipedia.org/wiki/Projections_onto_convex_sets). The method is illustrated by the figure below (of course you do not project on lines but on spaces having larger dimension).

![POCS](images/pocs.png "POCS")

### 1. Low-pass filter
Express the filtering operation by building the appropriate $N\times N$ matrix $P$ and check that it works using the following example.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
N = 100
w = 10 # cut-off frequency
k = np.arange(0, N)
w1 = 3
w2 = 7
w3 = 12

# generate simple signals
x1 = np.cos(2*w1*np.pi*k/N) + np.sin(2*w2*np.pi*k/N)
x2 = np.sin(2*w1*np.pi*k/N) + np.cos(2*w3*np.pi*k/N)

In [ ]:
# visualize the base signals
plt.plot(x1)
plt.plot(x2, 'r')

In [ ]:
W_hat = # insert the normalized Fourier matrix you implemented in the previous exercise session

In [ ]:
# Build the matrix P
P = # Your code here. 

In [ ]:
# Try the filtering on the test signals and make sure it behaves appropriately
X1 = W_hat@x1
X2 = W_hat@x2
x1_f = # filtered output
x2_f = # filtered output

NB: Make sure the filtered output is **real** (or its imaginary part should be smaller than $10^{-12}$).

In [ ]:
# Plot the filtered signals
# x1_f should still contain both frequencies, x2_f only one
plt.plot(x1_f)
plt.plot(x2_f, 'r')

Prove that $P$ is a projection.

### 2. Signal extension

In order to express the condition on $x[k]$ as a pure matrix operation we need to make use of an extension of the input signal and design a slightly different Fourier transform matrix to use properly those extended signals. 

Let us denote by $x_E$ the vector from $\mathbb{R}^m$ containing the known values of $x$, i.e. the $x_k$ for $k\in E$.
For each vector $y$ in $\mathbb{R}^N$ we define its extension as $\tilde{y} = \begin{pmatrix}y \\ x_E\end{pmatrix}$. This notation will remain throughout the notebook, i.e. a vector with a tilde denotes its extension made by adding the $x_E$ values at the end.

Let us define the matrix $B_E$ and $y=B_E x$, s.t. $y[k] = 0$ if $k\in E$ and $y[k] = x[k]$ otherwise. 

Write a function that returns its extended version $\tilde{B_E}$ s.t. $\tilde{y}=\tilde{B_E}\tilde{x}$, for any $x\in\mathbb{R}^N$. $\tilde{B_E}$ is a square matrix of size $N+M$.

In [ ]:
# E will be any list of indices, e.g. for N=5, a suitable E could be [1, 3]
def Bt_E(N, M, E):
    # your code here
    return None

Let us know design $C_E$ as an operator from $\mathbb{R}^{N+M} \rightarrow \mathbb{R}^{N+M}$ such that when applied to any extended vector $\tilde{z}$ s.t. $\tilde{z}[k] = 0, \forall k\in E$ as input (i.e. for instance the output of $\tilde{B}_E$), it generates a vector $\tilde{z}_R$ s.t.:

$\tilde{z}_R[k] = \left\{\begin{matrix} x_k \mbox{ if } k\in E \\ \tilde{z}[k] \mbox{ otherwise} \end{matrix}\right.$

Write a function that generates $C_E$. 

In [ ]:
# E will be any list of indices, e.g. for N=5, a suitable E could be [1, 3]
def C_E(N, M, E):
    # your code here
    return None

What is the effect of $D_E = C_E \tilde{B}_E$ on an extended signal $\tilde{x}$ ? 

Verify (numerically on an example) that $D_E$ is a projector.

In [ ]:
# your answer here

### 3. Extended signals in the Fourier domain
Let us now define an operator that will work almost as the normalized Fourier transform, except that it will be applied to the extended signals and preserve the $x_E$ part. This can be easily done using the following block matrix:

$\tilde{W} = \begin{pmatrix}\hat{W} & 0 \\0 & I_M\end{pmatrix}$.

Prove that it is orthonormal. 

Write a function that builds this matrix.

In [ ]:
def Wt_E(N, M, E):
    # your code here
    return None

Multiplying an extended signal $\tilde{x}$ by $\tilde{W}$ will result in a vector containing the Fourier transform of $x$ followed by $x_E$, preserving the known values.

Recalling the low-pass matrix $P$ defined previously, build $\tilde{P}$ s.t. when applied to $\tilde{W}\tilde{x}$ it results in a vector containing the filtered values (still in the Fourier domain) followed by $x_E$.

In [ ]:
def Pt_E(N, M, w):
    # your code here
    return None

A signal $\tilde{x}$ will be filtered by doing $\tilde{W}^H \tilde{P}\tilde{W}\tilde{x}$.
Prove that this is a projection.

### 4. Signal restoration

We can now use all defined above to implement a function that performs an iteration, taking as input the extension of $x$ as defined above, that does the following:
- compute the filtered version of the signal using $\tilde{W}^H \tilde{P}\tilde{W}$
- restore the known values in the signal by using $D_E = C_E\tilde{B}_E$

In [ ]:
def restoration_iter(Wt, Pt, Dt, xt):
    # your code here
    # make sure to force the result to real values by using np.real before returning
    return None

Finally we are ready to apply all this to an example.

In [ ]:
# Setup an example
N = 100
w = 10 # cut-off
w1 = 3
w2 = 7
E =  np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95])
# E =  np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90]) # try with less known points
M = len(E)
x = np.cos(2*w1*np.pi*np.arange(0, N)/N) + np.sin(2*w2*np.pi*np.arange(0,N)/N) # original signal
y = np.random.rand(N)
y[E] = x[E] # restore known values
xe = x[E]

In [ ]:
plt.plot(y) # plot the "acquired" signal

In [ ]:
plt.plot(x) # plot the ground truth signal

Generate $\tilde{y}$ (this only need to be done once)

In [ ]:
yt = # your code here

Run a number of iterations of the method and plot the result:

In [ ]:
def signal_restore(Wt, Pt, Dt, yt, niter=20):
    yr0 = yt # initialize
    for k in range(niter):
        yr1 = restoration_iter(Wt, Pt, Dt, yr0)
        yr0 = yr1
    return yr1

In [ ]:
Wt = Wt_E(N, M, E)
Dt = C_E(N, M, E)@Bt_E(N, M, E)
Pt = Pt_E(N, M, w)
yr = signal_restore(Wt, Pt, Dt, yt)

In [ ]:
plt.plot(yr[:N]) # plot reconstructed signal
plt.plot(x, 'r') # plot original for comparison

Depending on $M$, you will find that the method can reconstruct perfectly the signal or not.  